### Fake News Classifier using LSTM

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/fnc_train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/fnc_train.csv'

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label', axis=1)
y = df['label']

In [ ]:
X.shape

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
 voc_size = 5000

In [ ]:
# one hot representation

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords  # used to clean data (symbols, spaces)

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
one_hot_rep = [one_hot(words, voc_size) for words in corpus]

In [ ]:
len(max(one_hot_rep, key=len))

In [ ]:
##### Embedding Representation
sent_len = 20
Embedded_docs = pad_sequences(one_hot_rep, padding='pre', maxlen=sent_len)

In [ ]:
len(Embedded_docs)

In [ ]:
# model
embedded_vector_features = 40  #dimensions
model = Sequential()
model.add(Input(shape=(sent_len,)))
model.add(Embedding(voc_size, embedded_vector_features))
model.add(LSTM(100)) # one layer -100 neurons
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import numpy as np
X_final = np.array(Embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype('int')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)